In [1]:
    %use dataframe, kandy

In [2]:
USE {
    dependencies {
        implementation("org.apache.commons:commons-math3:3.6.1")
    }
}

In [3]:
import kotlin.io.path.*

fun DataFrame<*>.addConfigColumn(name: String): DataFrame<*> =
    add(column(List(rowsCount()) { name }).rename("config"))

fun DataFrame<*>.addProjectColumn(): DataFrame<*> =
    add(this["benchmark"].map { (it as String).substringBefore('-') }.rename("project"))

val folder = "../output/cache"

val data = Path(folder).listDirectoryEntries().map {
    DataFrame.readCSV((it / "smtData.csv").toFile()).addConfigColumn(it.name).addProjectColumn().sortBy("project")
}.concat()

data.columnNames()

[benchmark, cnt, sat, unsat, unknown, reusedUnsat, solvingTime, checkingTime, updatingTime, config, project]

In [4]:
data.filter { config == "kfull-tt" }
    .groupBy("project")
    .aggregate {
        val total = unsat.sum()
        val reused = reusedUnsat.sum()
        (total - reused).toDouble() / total into "unique unsat"
        reused.toDouble() / total into "reused unsat"
    }
    .gather("reused unsat", "unique unsat")
    .into("name", "value")
    .groupBy("name")
    .plot { 
        bars { 
            x("project")
            y("value")
            fillColor("name") {
                scale = categorical(
                    "unique unsat" to Color.GREY,
                    "reused unsat" to Color.ORANGE,
                )
            }
            position = Position.stack()
        }
    }

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="M4re0W"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"&merged_groups":["reused unsat","reused unsat","reused unsat","reused unsat","unique unsat","unique unsat","unique unsat","unique unsat"],
"name":["reused unsat","reused unsat","reused unsat","reused unsat","unique unsat","unique unsat","unique unsat","unique unsat"],
"project":["FASTJSON","GUAVA","SEATA","SPOON","FASTJSON","GUAVA","SEATA","SPOON"],
"value":[0.3322222222222222,0.22152466367713006,0.12790697674418605,0.7924376508447305,0.6677777777777778,0.7784753363228699,0.872093023255814,0.2075623491552695]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"values":["#a39999","#fc8452"],
"limits":["unique unsat","reused unsat"]
}],
"layers":[{
"mapping":{
"x":"project",
"y":"value",
"fill":"name",
"group":"&merged_groups"
},
"stat":"identity",
"sampling":"none",
"position":"stack",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("M4re0W");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 FASTJSON 
 
 
 
 
 
 
 
 
 GUAVA 
 
 
 
 
 
 
 
 
 SEATA 
 
 
 
 
 
 
 
 
 SPOON 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 
 
 
 
 0.2 
 
 
 
 
 
 
 0.4 
 
 
 
 
 
 
 0.6 
 
 
 
 
 
 
 0.8 
 
 
 
 
 
 
 1.0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 value 
 
 
 
 
 project 
 
 
 
 
 
 
 
 
 name 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 unique unsat 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 reused unsat

In [16]:
data.filter { config == "kfull-tt" }
    .groupBy("project", "config")
    .aggregate {
        (solvingTime.sum().toDouble() / cnt.sum()) into "solvingTime"
        (checkingTime.sum().toDouble() / cnt.sum()) into "checkingTime"
        (updatingTime.sum().toDouble() / cnt.sum()) into "updatingTime"
    }
    .gather("solvingTime", "checkingTime", "updatingTime")
    .into("type", "value")
    .groupBy("type")
    .plot {
        bars {
            x(project)
            y("value") {
                axis.name = "avg. time (ms)"
            }
            fillColor("type") {
                scale = categorical(
                    "solvingTime" to Color.ORANGE,
                    "checkingTime" to Color.GREY,
                    "updatingTime" to Color.BLUE,
                )
            }
            position = Position.stack()
        }
    }

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="UZdoMI"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"&merged_groups":["solvingTime","solvingTime","solvingTime","solvingTime","checkingTime","checkingTime","checkingTime","checkingTime","updatingTime","updatingTime","updatingTime","updatingTime"],
"project":["FASTJSON","GUAVA","SEATA","SPOON","FASTJSON","GUAVA","SEATA","SPOON","FASTJSON","GUAVA","SEATA","SPOON"],
"type":["solvingTime","solvingTime","solvingTime","solvingTime","checkingTime","checkingTime","checkingTime","checkingTime","updatingTime","updatingTime","updatingTime","updatingTime"],
"value":[350.8719927371766,515.3255907088507,69.08552631578948,344.2914349276974,0.6836132546527462,56.39767721265519,0.0,0.41212458286985537,0.0,0.0,0.0,0.0]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"name":"avg. time (ms)",
"limits":[null,null]
},{
"aesthetic":"fill",
"values":["#fc8452","#a39999","#5470c6"],
"limits":["solvingTime","checkingTime","updatingTime"]
}],
"layers":[{
"mapping":{
"x":"project",
"y":"value",
"fill":"type",
"group":"&merged_groups"
},
"stat":"identity",
"sampling":"none",
"position":"stack",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("UZdoMI");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 FASTJSON 
 
 
 
 
 
 
 
 
 GUAVA 
 
 
 
 
 
 
 
 
 SEATA 
 
 
 
 
 
 
 
 
 SPOON 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 avg. time (ms) 
 
 
 
 
 project 
 
 
 
 
 
 
 
 
 type 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 solvingTime 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 checkingTime 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 updatingTime

In [15]:
data.filter { config == "pure-tt" }
    .plot {
        boxplot(
            project,
            column<Double>(
                solvingTime.values.zip(cnt.values).map { it.first.toDouble() / it.second }
            ).named("avg. solving time (ms)")
        )
    }

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="CmD4X3"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"project",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"avg. solving time (ms)",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"min",
"lower":"lower",
"middle":"middle",
"upper":"upper",
"ymax":"max",
"y":"y"
},
"stat":"identity",
"data":{
"middle":[73.73746987951807,305.7127536231884,54.714285714285715,848.75],
"min":[0.30973451327433627,58.56576200417537,3.0,108.25],
"max":[1169.225,961.4852071005918,323.44444444444446,2215.9333333333334],
"upper":[557.7965125280623,618.0731330713082,218.69444444444446,1509.9565217391305],
"lower":[3.056241460081983,132.63204133327972,6.625,238.77464788732394],
"x":["FASTJSON","GUAVA","SEATA","SPOON"]
},
"sampling":"none",
"position":"identity",
"geom":"boxplot"
},{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":["FASTJSON","FASTJSON","GUAVA"],
"y":[1391.0298507462687,1769.4117647058824,1380.6746203904556]
},
"sampling":"none",
"position":"identity",
"geom":"point"
}]
};
 var plotContainer = document.getElementById("CmD4X3");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 FASTJSON 
 
 
 
 
 
 
 
 
 GUAVA 
 
 
 
 
 
 
 
 
 SEATA 
 
 
 
 
 
 
 
 
 SPOON 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 1,500 
 
 
 
 
 
 
 2,000 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 avg. solving time (ms) 
 
 
 
 
 project